# Exercise 11

## Group ID: 
*   Person1
*   Person2
*   Person3

## Exercise day: Tuesday/Wednesday






## Description
In this exercise you will implement a Tokenizer class that will be used to tokenize a given text.
A token will be a short sequence of bytes and tokenization will be about splitting up the text into those short substrings and then representing it in terms of token ids.
This allows language models to work on a better text representation than just using a character level model as in the last exercise.
We will use the so called "Byte Pair Encoding" (BPE) algorithm to tokenize the text. The BPE algorithm is a simple algorithm that iteratively merges the most frequent pair of bytes/ids in the text. This notebook will guide you through the implementation of the Tokenizer class. To make the representation of the text easier, we will use integer ids to represent the bytes of the text.

Note: The tokenizer is not part of the model, it is a preprocessing step that is used to tokenize the text before feeding it to the model. Therefore, it can be trained on a different dataset than the model.
Once trained, the tokenizer is capable of encoding and decoding any given text to/from a list of tokens.

## Tasks
1. Implement the `get_stats` method that will return a dictionary containing the frequency of each pair of characters in the text. (0.5 points)
1. Implement the `merge` method that will merge the most frequent pair of characters in the text. (0.5 points)
1. Implement the `fit` method that will train the tokenizer on the given text. (1 point)
1. Implement the `encode` method that will encode the given text to a list of tokens. (1 point)
1. Implement the `decode` method that will decode the given list of tokens to a text. (1 point)
1. Implement the `BPE` class. This class should contain the previously implemented methods. (1 point)

In [146]:
def get_stats(ids):
    """ Returns a dictionary with the number of times each pair of ids appears in the input list """
    pass

In [147]:
ids = [1, 2, 3, 1, 2, 3, 1, 2, 3]
expected = {(1, 2): 3, (2, 3): 3, (3, 1): 2}
assert get_stats(ids) == expected

In [148]:
def merge(ids, pair, new_id):
    """Merge every pair of elements in ids that are equal to pair into a single element new_id"""
    pass

In [149]:
ids = [0,1,2,3,0,1,2,3]
pair = (0,1)
new_id = 4
new_ids = merge(ids, pair, new_id)
expected = [4,2,3,4,2,3]
assert new_ids == expected, f"Merge failed new_ids actual: {new_ids} expected: {expected}"

In [150]:
def fit(ids, max_iter=1000):
    """Fit the model to the data by merging recursively the most common pairs of ids max_iter times or until no pair appears more than once. If two pairs have the same frequency, the one that appears first is chosen.
    Returns the fitted model and a dictionary with the merges containing the new_id for each pair.
    To ensure each new_id is unique (and our results are comparable), it is set to the maximum id in the list plus one.
    """
    pass

In [151]:
ids = [0,1,2,3,0,1,2,3]
num_iterations = 1
new_ids, merges = fit(ids,num_iterations)
assert new_ids == [4,2,3,4,2,3], f"Wrong ids after fit {new_ids}"
assert merges == {(0,1):4}, f"Wrong merges after fit {merges}"

In [152]:
ids = [0,1,2,3,0,1,2,3]
num_iterations = 2
new_ids, merges = fit(ids,num_iterations)
assert new_ids == [5,3,5,3], f"Wrong ids after fit {new_ids}"
assert merges == {(0,1):4, (4,2):5}, f"Wrong merges after fit {merges}"

In [153]:
def encode(ids, merges):
    """Encode the input list of ids using the merges dictionary"""
    pass

In [154]:
ids = [0,1,2,3,0,1,2,3]
merges = {(0,1):4}
encoded_ids = encode(ids, merges)
expected = [4,2,3,4,2,3]
assert encoded_ids == expected, f"Wrong encoded ids {encoded_ids} expected: {expected}"

In [155]:
def decode(ids, merges):
    """Decode the input list of ids using the merges dictionary"""
    pass


In [ ]:
ids = [6,6]
merges = {(0,1):4, (4,2):5, (5,3):6}
decoded_ids = decode(ids, merges)
expected = [0,1,2,3,0,1,2,3]
assert decoded_ids == expected, f"Wrong decoded ids {decoded_ids} expected: {expected}"

In [156]:
class BPE:
    def __init__(self, max_iter:int=1000):
        pass

    def fit(self, text):
        pass

    def encode(self, text):
        pass

    def decode(self, ids):
        pass


In [157]:
text = "A large language model (LLM) is a type of computational model designed for natural language processing tasks such as language generation. As language models, LLMs acquire these abilities by learning statistical relationships from vast amounts of text during a self-supervised and semi-supervised training process. The largest and most capable LLMs are artificial neural networks built with a decoder-only transformer-based architecture, enabling efficient processing and generation of large-scale text data. Modern models can be fine-tuned for specific tasks or guided by prompt engineering. These models acquire predictive power regarding syntax, semantics, and ontologies inherent in human language corpora, but they also inherit inaccuracies and biases present in the data they are trained in." # cleaned version of the first paragraph of the Wikipedia page on LLMs
ids = list(map(int, text.encode('utf-8')))

In [158]:
bpe = BPE(max_iter=1000)
bpe.fit(text)
encoded_text = bpe.encode(text)
decoded_text = bpe.decode(encoded_text)
assert text == decoded_text, "Decoding failed"

## Evaluation: 

Let us check what a tokenizer will give us for training a simple language model.

We will use the tiny Shakespeare dataset and train with and without tokenization and compare generated text.


In [ ]:
!wget 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(device)

In [161]:
# Compute tokenizer
with open('input.txt') as f:
    text = f.read()
bpe = BPE(max_iter=1000)
bpe.fit(text)
enc = bpe.encode(text)

In [ ]:
print(enc[:1000])

In [ ]:
print(bpe.decode(enc[:200]))

In [164]:
class TextDataset(Dataset):
    def __init__(self, enc_text, blocksize):
        self.data = enc_text
        self.blocksize = blocksize

    def __len__(self):
        return len(self.data)-self.blocksize-1

    def __getitem__(self, idx):
        x = self.data[idx:idx+self.blocksize]
        y = self.data[idx+self.blocksize]
        return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long), 

In [165]:
class MLPNextTokenPredictor(nn.Module):
    def __init__(self, vocab_size, block_size, embed_dim=32, hidden_dim=128):
        super().__init__()
        self.vocab_size = vocab_size
        self.block_size = block_size
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim

        self.emb = nn.Embedding(vocab_size, embed_dim)

        self.fc1 = nn.Linear(block_size * embed_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        emb = self.emb(x)
        emb_flat = emb.view(-1, self.block_size * self.embed_dim)

        h = torch.relu(self.bn1(self.fc1(emb_flat)))
        h = torch.relu(self.bn2(self.fc2(h)))
        logits = self.fc3(h)

        return logits

In [166]:
# generate new text based on continuing provided one
def generate(model, block_size, starting_text):
    model.eval()
    assert len(starting_text) >= block_size
    x = torch.tensor(starting_text, dtype=torch.long).to(device)

    with torch.no_grad():
        for _ in range(100):
            logits = model(x[-block_size:])
            next_token = torch.multinomial(F.softmax(logits, dim=-1), 1)
            starting_text = starting_text + [next_token.item()]
            x = torch.cat([x, next_token.squeeze(0)])
    return starting_text

In [167]:
def produce_example_text(model, block_size, tokenizer):
    starting_text = """We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance;'
"""
    tokenized_starting_text = tokenizer.encode(starting_text)
    continuation = generate(model, block_size, tokenized_starting_text)
    return tokenizer.decode(continuation[len(tokenized_starting_text):])

In [ ]:
# Compare with untrained MLP:
vocab_size = max(bpe.vocab)+1
block_size = 32
mlp_untrained = MLPNextTokenPredictor(vocab_size, block_size)
mlp_untrained.to(device)
continuation = produce_example_text(mlp_untrained, block_size, bpe)
print(continuation)

In [169]:
def train(model, dataloader, optimizer, nr_epochs):
    model.train()
    for epoch in range(nr_epochs):
        avg_loss = 0.0
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(x)
            loss = F.cross_entropy(logits.view(-1, model.vocab_size), y.view(-1))
            loss.backward()
            optimizer.step()
            avg_loss += loss.item() / len(dataloader)
        print(f"Epoch {epoch} loss: {avg_loss}")

In [ ]:
# training on tokenized text
block_size = 32
vocab_size = max(bpe.vocab)+1
nr_epochs = 15

dataloader = DataLoader(TextDataset(enc, block_size), batch_size=32, shuffle=True, drop_last=True)
mlp_tokenizer = MLPNextTokenPredictor(vocab_size, block_size)
mlp_tokenizer.to(device)
optimizer = optim.AdamW(mlp_tokenizer.parameters(), lr=0.001, weight_decay=0.01)

train(mlp_tokenizer, dataloader, optimizer, nr_epochs)


In [ ]:
# generate new text using tokenization based MLP
continuation = produce_example_text(mlp_tokenizer, block_size, bpe)
print(continuation)

In [ ]:
# training on original text
block_size = 64 # let us give more context because of no tokenization
# convert text to numbers
char_enc = [ord(c) for c in text]
vocab_size = max(char_enc) + 1
nr_epochs = 15

dataloader = DataLoader(TextDataset(char_enc, block_size), batch_size=32, shuffle=True, drop_last=True)
mlp_wo_tokenizer = MLPNextTokenPredictor(vocab_size, block_size)
mlp_wo_tokenizer.to(device)
optimizer = optim.AdamW(mlp_wo_tokenizer.parameters(), lr=0.001, weight_decay=0.01)

train(mlp_wo_tokenizer, dataloader, optimizer, nr_epochs)

In [ ]:
# generate new text using non-tokenization based MLP
class no_tok:
    def encode(self, text):
        return [ord(c) for c in text]

    def decode(self, ids):
        return ''.join([chr(i) for i in ids])

t = produce_example_text(mlp_wo_tokenizer, block_size, no_tok())
print(t)